### Selected model is noaug-grad, here are the results.

In [1]:
from torch import nn
import pretrainedmodels
import pandas as pd
import torch
import numpy as np

In [2]:
#load xception model
import pretrainedmodels

model_name = 'xception' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
#add last layer with 3 classes
model.last_linear = nn.Sequential(nn.Linear(2048,3),nn.Softmax())
#put model on gpu
model.cuda()
#load trained model with weights
model.load_state_dict(torch.load('torch_updated_trained_model.pt'))

RuntimeError: Error(s) in loading state_dict for Xception:
	Missing key(s) in state_dict: "last_linear.0.weight", "last_linear.0.bias". 
	Unexpected key(s) in state_dict: "last_linear.3.weight", "last_linear.3.bias", "last_linear.1.weight", "last_linear.1.bias". 

In [ ]:
#create empty pandas data frame with specified colunm names.
results = pd.DataFrame(columns=['Id','task_1','task_2'])


In [ ]:
#load test images
from torchvision import datasets
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
#resize,crop,transfer to pytorch tensor and normalize the images
transform = transforms.Compose([transforms.Resize(299),
                                     transforms.CenterCrop(299),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])                                        
test_data = datasets.ImageFolder(root = 'data/test',transform = transform)
test_loader = DataLoader(test_data,batch_size = 1,shuffle = False,num_workers = 0)
print(test_data.class_to_idx)

In [46]:
#get all pictures id's
import glob
test_dir = glob.glob('data/*/*/*')
#add to pandas dataframe
results['Id'] = test_dir
print(results)

Id task_1 task_2
0                data\test\melanoma\ISIC_0012258.jpg    NaN    NaN
1                data\test\melanoma\ISIC_0012356.jpg    NaN    NaN
2                data\test\melanoma\ISIC_0012369.jpg    NaN    NaN
3                data\test\melanoma\ISIC_0012395.jpg    NaN    NaN
4                data\test\melanoma\ISIC_0012425.jpg    NaN    NaN
..                                               ...    ...    ...
595  data\test\seborrheic_keratosis\ISIC_0014647.jpg    NaN    NaN
596  data\test\seborrheic_keratosis\ISIC_0014648.jpg    NaN    NaN
597  data\test\seborrheic_keratosis\ISIC_0014649.jpg    NaN    NaN
598  data\test\seborrheic_keratosis\ISIC_0014652.jpg    NaN    NaN
599  data\test\seborrheic_keratosis\ISIC_0014653.jpg    NaN    NaN

[600 rows x 3 columns]


In [47]:
#define criterion
criterion = nn.CrossEntropyLoss()


In [48]:
#make predictions
task_1 = []
task_2 = []
def test(loaders, model, criterion, use_cuda):
    #make list probalities for melanoma (task_1) and seborrheic keratosis (task_2)
    
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders):
        with torch.no_grad():
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            #extract melanoma and seborrheic_keratosis_proba probabilities
            melanoma_proba = output[0].tolist()[0]
            seborrheic_keratosis_proba = output[0].tolist()[1]
            
            #append melanoma probablilies
            task_1.append(melanoma_proba)
            #append seborrheic_keratosis probablilies
            task_2.append(seborrheic_keratosis_proba)
       

            

            # calculate the loss
            loss = criterion(output, target)
            # update average test loss 
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(test_loader, model, criterion, True)

Test Loss: 0.816364


Test Accuracy: 74% (447/600)


In [49]:
#add those lists as entries to our task_1 and task_2 columns

results['task_1'] = task_1
results['task_2'] = task_2
print(results)

Id    task_1    task_2
0                data\test\melanoma\ISIC_0012258.jpg  0.075565  0.361669
1                data\test\melanoma\ISIC_0012356.jpg  0.057701  0.449189
2                data\test\melanoma\ISIC_0012369.jpg  0.267915  0.721688
3                data\test\melanoma\ISIC_0012395.jpg  0.197210  0.772461
4                data\test\melanoma\ISIC_0012425.jpg  0.024590  0.371998
..                                               ...       ...       ...
595  data\test\seborrheic_keratosis\ISIC_0014647.jpg  0.008840  0.287305
596  data\test\seborrheic_keratosis\ISIC_0014648.jpg  0.000032  0.000286
597  data\test\seborrheic_keratosis\ISIC_0014649.jpg  0.000018  0.000214
598  data\test\seborrheic_keratosis\ISIC_0014652.jpg  0.101073  0.306192
599  data\test\seborrheic_keratosis\ISIC_0014653.jpg  0.000079  0.000211

[600 rows x 3 columns]


In [50]:
#convert the pandas data frame to a csv
results.to_csv('predictions.csv')

# Results
The model performed extremely well in categorizing bening lesions as benign, but extremely poor at classifying melanoma correctly. This is extremely concerning as the algorithm could be missing patients who have cancer.
I will attempt to improve the model by implementing more data with positive melanoma.


<img src="results/first_try.png" alt="first_try" width="500"/>                <img src="results/first_try_confusion_matrix.py.png" alt="first_try_confusion_matrix" width="500"/>                               
